In [22]:
import sqlite3
import pandas as pd
import polars as pl
conn = sqlite3.connect("IMDB.db")
cursor = conn.cursor()

In [23]:
from datetime import datetime

now = datetime.now()
print("Compilado em:", now.strftime("%d/%m/%Y %H:%M:%S"))

Compilado em: 21/10/2025 21:58:17


# IMDB Querys - LAB 13 (ME315)
## Analisando estrutura das tabelas presentes no banco de dados

**Tabela basics**:

In [33]:
head_basics = pl.read_database(
    "SELECT * FROM basics LIMIT 6;",
    connection= conn
)
head_basics

tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
str,str,str,str,i64,f64,null,f64,str
"""tt0000001""","""short""","""Carmencita""","""Carmencita""",0,1894.0,null,1.0,"""Documentary,Short"""
"""tt0000002""","""short""","""Le clown et ses chiens""","""Le clown et ses chiens""",0,1892.0,null,5.0,"""Animation,Short"""
"""tt0000003""","""short""","""Pauvre Pierrot""","""Pauvre Pierrot""",0,1892.0,null,5.0,"""Animation,Comedy,Romance"""
"""tt0000004""","""short""","""Un bon bock""","""Un bon bock""",0,1892.0,null,12.0,"""Animation,Short"""
"""tt0000005""","""short""","""Blacksmith Scene""","""Blacksmith Scene""",0,1893.0,null,1.0,"""Comedy,Short"""
"""tt0000006""","""short""","""Chinese Opium Den""","""Chinese Opium Den""",0,1894.0,null,1.0,"""Short"""


**Tabela principals**:

In [32]:
head_principals = pl.read_database(
    "SELECT * FROM principals LIMIT 6;",
    connection=conn
)
head_principals

tconst,ordering,nconst,category,job,characters
str,i64,str,str,str,str
"""tt0000001""",1,"""nm1588970""","""self""",null,"""[Self]"""
"""tt0000001""",2,"""nm0005690""","""director""",null,null
"""tt0000001""",3,"""nm0005690""","""producer""","""producer""",null
"""tt0000001""",4,"""nm0374658""","""cinematographer""","""director of photography""",null
"""tt0000002""",1,"""nm0721526""","""director""",null,null
"""tt0000002""",2,"""nm1335271""","""composer""",null,null


**Tabela ratings**:

In [31]:
head_ratings = pl.read_database(
    "SELECT * FROM ratings LIMIT 6;",
    connection=conn
)
head_ratings

tconst,averageRating,numVotes
str,f64,i64
"""tt0000001""",5.7,2092
"""tt0000002""",5.6,283
"""tt0000003""",6.5,2099
"""tt0000004""",5.4,183
"""tt0000005""",6.2,2835
"""tt0000006""",5.0,196


# Quais são os 5 filmes com as maiores notas (averageRating)?

In [29]:
query = """
SELECT DISTINCT titletype
FROM basics
"""

tipos = pl.read_database(query, connection=conn)
tipos

titleType
str
"""short"""
"""movie"""
"""tvShort"""
"""tvMovie"""
"""tvEpisode"""
…
"""tvMiniSeries"""
"""tvSpecial"""
"""video"""


In [30]:
query = """
SELECT b.primaryTitle, r.averageRating, r.numVotes
FROM ratings r
JOIN basics b ON r.tconst = b.tconst
WHERE b.titletype = 'movie'
ORDER BY r.averageRating DESC, r.numVotes DESC
LIMIT 5;
"""

filmes_melhores_avaliados = pl.read_database(query, connection=conn)
filmes_melhores_avaliados


primaryTitle,averageRating,numVotes
str,f64,i64
"""Kaveri""",10.0,1023
"""Kurukku""",10.0,451
"""Jedal Dar Omghe 30 Metri""",10.0,142
"""Sargashte""",10.0,134
"""Gorgeous Rascal""",10.0,115


Os 5 filmes melhor avaliados estão com nota 10. Sendo esses:

- **Kaveri**
- **Kurukku**
- **Jedal Dar Omghe 30 Metri**
- **Sargashte**
- **Gorgeous Rascal**


Porém, é perceptível que todos têm numero de avaliações baixas (tendo em vista que o IMDB é uma plataforma internacional). Logo, para uma seleção mais confiável, será utilizado um critério para evitar esse cenário.

In [34]:
query = """
SELECT b.primaryTitle, r.averageRating, r.numVotes
FROM ratings r
JOIN basics b ON r.tconst = b.tconst
WHERE b.titletype = 'movie'
AND r.numVotes > 100000
ORDER BY r.averageRating DESC, r.numVotes DESC
LIMIT 5;
"""

filmes_melhores_avaliados_com_criterio = pl.read_database(query, connection=conn)
filmes_melhores_avaliados_com_criterio

primaryTitle,averageRating,numVotes
str,f64,i64
"""The Shawshank Redemption""",9.3,2948622
"""The Godfather""",9.2,2055335
"""The Dark Knight""",9.0,2929338
"""The Lord of the Rings: The Ret…",9.0,2018311
"""Schindlers List""",9.0,1479154


Após o uso de 'numVotes' > 100.000 , os filmes melhores avaliados são:
1. **The Shawshank Redemption** (Nota: 9.3)
2. **The Godfather** (Nota: 9.2)
3. **The Dark Knight** (Nota: 9.0)
4. **The Lord of the Rings: The Return of the King** (Nota: 9.0)
5. **Schindlers List** (Nota: 9.0)

# Qual é o gênero mais frequente entre os filmes com nota maior que 8?

### Analisando os gêneros presentes na tabela

In [35]:
query = """
SELECT DISTINCT genres
FROM basics;
"""

generos_basics = pl.read_database(query, connection=conn)
generos_basics

genres
str
"""Documentary,Short"""
"""Animation,Short"""
"""Animation,Comedy,Romance"""
"""Comedy,Short"""
"""Short"""
…
"""Adult,Short,Thriller"""
"""Adult,Animation,Talk-Show"""
"""Biography,Crime,Fantasy"""


Buscando gênero mais frequente em notas maiores que 8:

In [38]:
# Método mais idiomático do Polars
all_genres = (
    high_rating
    .filter(pl.col("genres").is_not_null())
    .with_columns(
        pl.col("genres")
        .str.split(",")
        .alias("genre_list")
    )
    .explode("genre_list")
    .group_by("genre_list")
    .agg(pl.len().alias("count"))
    .sort("count", descending=True)
    .head(2)
)

print(all_genres)

shape: (2, 2)
┌─────────────┬───────┐
│ genre_list  ┆ count │
│ ---         ┆ ---   │
│ str         ┆ u32   │
╞═════════════╪═══════╡
│ Documentary ┆ 10564 │
│ Drama       ┆ 8114  │
└─────────────┴───────┘


Logo, o gênero com maior contagem de notas atribuídas maiores que 8 é **Documentary**. Este fenômeno ocorre porque documentários, com menos avaliações e públicos mais nichados, mantêm notas altas mais facilmente. Enquanto filmes de outros gêneros recebem volume massivo de votos com maior variação, documentários beneficiam-se de engajamento mais favorável entre seu público específico.

# Quais são os 3 atores/atrizes que mais participaram de filmes com nota maior que 7.5?

**Contagem de atores**

In [39]:
query = """
SELECT COUNT(DISTINCT nconst)
FROM principals
WHERE characters IS NOT NULL;
"""

atores = pl.read_database(query, connection=conn)
atores

COUNT(DISTINCT nconst)
i64
4069682


**Id dos Filmes com averageRating > 7.5**

In [40]:
query = """
SELECT r.tconst
FROM ratings r
JOIN basics b ON r.tconst = b.tconst
WHERE r.averageRating > 7.5
AND b.titleType = 'movie';
"""

filmes = pl.read_database(query, connection=conn)
filmes

tconst
str
"""tt0002305"""
"""tt0003386"""
"""tt0003748"""
"""tt0003771"""
"""tt0004167"""
…
"""tt9911774"""
"""tt9914642"""
"""tt9914644"""


In [41]:
query = """
SELECT nconst, COUNT(tconst) AS Total
FROM principals
WHERE tconst IN (SELECT r.tconst
                 FROM ratings r
                 JOIN basics b ON r.tconst = b.tconst
                 WHERE r.averageRating > 7.5
                 AND b.titleType = 'movie')
GROUP BY nconst
ORDER BY Total DESC 
LIMIT 3
"""

atores = pl.read_database(query, connection=conn)
atores

nconst,Total
str,i64
"""nm0004660""",233
"""nm5954636""",200
"""nm0595934""",170


Como não foi fornecido o nome dos atores, foi selecionado os ID's dos 3 autores.
- **nm0004660** (233 participações)
- **nm5954636** (200 participações)
- **nm0595934** (170 participações)

In [42]:
conn.close()